In [1]:
import sys
import os
import logging
import papermill as pm
import scrapbook as sb
from tempfile import TemporaryDirectory
import numpy as np
import tensorflow.compat.v1 as tf
tf.get_logger().setLevel('ERROR') # only show error messages

if os.path.join('..', '..', 'recommenders') not in sys.path:
    sys.path.append(os.path.join('..', '..', 'recommenders'))

from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.models.deeprec.deeprec_utils import (
    prepare_hparams
)
from recommenders.datasets.amazon_reviews import download_and_extract, data_preprocessing
from recommenders.datasets.download_utils import maybe_download


# Locally import the model
from models.deeprec.models.sequential.din import DIN_RECModel as SeqModel


#from recommenders.models.deeprec.models.sequential.nextitnet import NextItNetModel

from recommenders.models.deeprec.io.sequential_iterator import SequentialIterator
#from recommenders.models.deeprec.io.nextitnet_iterator import NextItNetIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

import pickle as pkl
from recommenders.models.deeprec.deeprec_utils import load_dict

System version: 3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]
Tensorflow version: 2.9.0


In [2]:
yaml_file = '../../recommenders/models/deeprec/config/din.yaml'  

In [3]:
EPOCHS = 10
BATCH_SIZE = 400
RANDOM_SEED = SEED  # Set None for non-deterministic result

data_path = os.path.join("..", "..", "tests", "resources", "deeprec", "slirec")

In [4]:

# for test
train_file = os.path.join(data_path, r'train_data')
valid_file = os.path.join(data_path, r'valid_data')
test_file = os.path.join(data_path, r'test_data')
user_vocab = os.path.join(data_path, r'user_vocab.pkl')
item_vocab = os.path.join(data_path, r'item_vocab.pkl')
cate_vocab = os.path.join(data_path, r'category_vocab.pkl')
output_file = os.path.join(data_path, r'output.txt')

reviews_name = 'reviews_Movies_and_TV_5.json'
meta_name = 'meta_Movies_and_TV.json'
reviews_file = os.path.join(data_path, reviews_name)
meta_file = os.path.join(data_path, meta_name)
train_num_ngs = 4 # number of negative instances with a positive instance for training
valid_num_ngs = 4 # number of negative instances with a positive instance for validation
test_num_ngs = 9 # number of negative instances with a positive instance for testing
# sample_rate = 0.01 # sample a small item set for training and testing here for fast example
sample_rate = 1

input_files = [reviews_file, meta_file, train_file, valid_file, test_file, user_vocab, item_vocab, cate_vocab]

if not os.path.exists(train_file):
    download_and_extract(reviews_name, reviews_file)
    download_and_extract(meta_name, meta_file)
    data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)
    #### uncomment this for the NextItNet model, because it does not need to unfold the user history
    # data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs, is_history_expanding=False)

# data_preprocessing(*input_files, sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)

user_vocab: 123960

item_vocab: 50024

cate_vocab: 164

hist_length:

In [5]:
hparams = prepare_hparams(yaml_file, 
                          embed_l2=0., 
                          layer_l2=0., 
                          learning_rate=0.001,  # set to 0.01 if batch normalization is disable
                          epochs=EPOCHS,
                          batch_size=BATCH_SIZE,
                          show_step=1,
                          MODEL_DIR=os.path.join(data_path, "model/din_dice/"),
                          SUMMARIES_DIR=os.path.join(data_path, "summary/din_dice/"),
                          user_vocab=user_vocab,
                          item_vocab=item_vocab,
                          cate_vocab=cate_vocab,
                          need_sample=True,
                          train_num_ngs=train_num_ngs, # provides the number of negative instances for each positive instance for loss computation.
#                           attention_mode="inner_product",
#                           activation=['dice', 'dice'],
#                           dice_momentum=0.9
            )

In [6]:
hparams.values()

{'use_entity': True,
 'use_context': True,
 'cross_activation': 'identity',
 'user_dropout': True,
 'dropout': [0.3, 0.3],
 'attention_dropout': 0.0,
 'load_saved_model': False,
 'fast_CIN_d': 0,
 'use_Linear_part': False,
 'use_FM_part': False,
 'use_CIN_part': False,
 'use_DNN_part': False,
 'init_method': 'tnormal',
 'init_value': 0.01,
 'embed_l2': 0.0,
 'embed_l1': 0.0,
 'layer_l2': 0.0,
 'layer_l1': 0.0,
 'cross_l2': 0.0,
 'cross_l1': 0.0,
 'reg_kg': 0.0,
 'learning_rate': 0.001,
 'lr_rs': 1,
 'lr_kg': 0.5,
 'kg_training_interval': 5,
 'max_grad_norm': 2,
 'is_clip_norm': 0,
 'dtype': 32,
 'optimizer': 'adam',
 'epochs': 10,
 'batch_size': 400,
 'enable_BN': True,
 'show_step': 1,
 'save_model': True,
 'save_epoch': 1,
 'write_tfevents': True,
 'train_num_ngs': 4,
 'need_sample': True,
 'embedding_dropout': 0.0,
 'EARLY_STOP': 10,
 'min_seq_length': 1,
 'slots': 5,
 'cell': 'SUM',
 'user_vocab': '../../tests/resources/deeprec/slirec/user_vocab.pkl',
 'item_vocab': '../../tests/re

In [7]:
input_creator = SequentialIterator
#### uncomment this for the NextItNet model, because it needs a special data iterator for training
#input_creator = NextItNetIterator

In [8]:
model = SeqModel(hparams, input_creator, seed=RANDOM_SEED)

with Timer() as train_time:
    model = model.fit(train_file, valid_file, valid_num_ngs=valid_num_ngs) 

# valid_num_ngs is the number of negative lines after each positive line in your valid_file 
# we will evaluate the performance of model on valid_file every epoch
print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))

/home/caozheng/Desktop/code/recommenders/examples/00_quick_start/../../recommenders/models/deeprec/models/base_model.py:755: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  curr_hidden_nn_layer = tf.compat.v1.layers.batch_normalization(
2022-06-18 08:31:27.187851: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 08:31:27.193263: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 08:31:27.193446: I tensorflow/stream_executor/cuda

step 1 , total_loss: 1.6098, data_loss: 1.6098
step 2 , total_loss: 1.6099, data_loss: 1.6099
step 3 , total_loss: 1.6073, data_loss: 1.6073
step 4 , total_loss: 1.6062, data_loss: 1.6062
step 5 , total_loss: 1.6026, data_loss: 1.6026
step 6 , total_loss: 1.6010, data_loss: 1.6010
step 7 , total_loss: 1.5959, data_loss: 1.5959
step 8 , total_loss: 1.5902, data_loss: 1.5902
step 9 , total_loss: 1.5876, data_loss: 1.5876
step 10 , total_loss: 1.5801, data_loss: 1.5801
step 11 , total_loss: 1.5822, data_loss: 1.5822
step 12 , total_loss: 1.5693, data_loss: 1.5693
step 13 , total_loss: 1.5524, data_loss: 1.5524
step 14 , total_loss: 1.5701, data_loss: 1.5701
step 15 , total_loss: 1.5560, data_loss: 1.5560
step 16 , total_loss: 1.5555, data_loss: 1.5555
step 17 , total_loss: 1.5353, data_loss: 1.5353
step 18 , total_loss: 1.5507, data_loss: 1.5507
step 19 , total_loss: 1.5180, data_loss: 1.5180
step 20 , total_loss: 1.5440, data_loss: 1.5440
step 21 , total_loss: 1.5171, data_loss: 1.5171
s

step 171 , total_loss: 0.2661, data_loss: 0.2661
step 172 , total_loss: 0.2615, data_loss: 0.2615
step 173 , total_loss: 0.2651, data_loss: 0.2651
step 174 , total_loss: 0.2884, data_loss: 0.2884
step 175 , total_loss: 0.2563, data_loss: 0.2563
step 176 , total_loss: 0.2590, data_loss: 0.2590
step 177 , total_loss: 0.2658, data_loss: 0.2658
step 178 , total_loss: 0.2448, data_loss: 0.2448
step 179 , total_loss: 0.2248, data_loss: 0.2248
step 180 , total_loss: 0.2561, data_loss: 0.2561
step 181 , total_loss: 0.2501, data_loss: 0.2501
step 182 , total_loss: 0.3357, data_loss: 0.3357
step 183 , total_loss: 0.2462, data_loss: 0.2462
step 184 , total_loss: 0.2399, data_loss: 0.2399
step 185 , total_loss: 0.2363, data_loss: 0.2363
step 186 , total_loss: 0.2496, data_loss: 0.2496
step 187 , total_loss: 0.2317, data_loss: 0.2317
step 188 , total_loss: 0.2319, data_loss: 0.2319
step 189 , total_loss: 0.2008, data_loss: 0.2008
step 190 , total_loss: 0.2040, data_loss: 0.2040
step 191 , total_los

step 340 , total_loss: 0.0723, data_loss: 0.0723
step 341 , total_loss: 0.0681, data_loss: 0.0681
step 342 , total_loss: 0.0587, data_loss: 0.0587
step 343 , total_loss: 0.0705, data_loss: 0.0705
step 344 , total_loss: 0.0578, data_loss: 0.0578
step 345 , total_loss: 0.0634, data_loss: 0.0634
step 346 , total_loss: 0.0699, data_loss: 0.0699
step 347 , total_loss: 0.0666, data_loss: 0.0666
step 348 , total_loss: 0.0667, data_loss: 0.0667
step 349 , total_loss: 0.0626, data_loss: 0.0626
step 350 , total_loss: 0.0586, data_loss: 0.0586
step 351 , total_loss: 0.0639, data_loss: 0.0639
step 352 , total_loss: 0.0550, data_loss: 0.0550
step 353 , total_loss: 0.0616, data_loss: 0.0616
step 354 , total_loss: 0.0629, data_loss: 0.0629
step 355 , total_loss: 0.0522, data_loss: 0.0522
step 356 , total_loss: 0.0616, data_loss: 0.0616
step 357 , total_loss: 0.0610, data_loss: 0.0610
step 358 , total_loss: 0.0565, data_loss: 0.0565
step 359 , total_loss: 0.0571, data_loss: 0.0571
step 360 , total_los

step 509 , total_loss: 0.0287, data_loss: 0.0287
step 510 , total_loss: 0.0256, data_loss: 0.0256
step 511 , total_loss: 0.0275, data_loss: 0.0275
step 512 , total_loss: 0.0263, data_loss: 0.0263
step 513 , total_loss: 0.0333, data_loss: 0.0333
step 514 , total_loss: 0.0323, data_loss: 0.0323
step 515 , total_loss: 0.0302, data_loss: 0.0302
step 516 , total_loss: 0.0254, data_loss: 0.0254
step 517 , total_loss: 0.0325, data_loss: 0.0325
step 518 , total_loss: 0.0316, data_loss: 0.0316
step 519 , total_loss: 0.0276, data_loss: 0.0276
step 520 , total_loss: 0.0276, data_loss: 0.0276
step 521 , total_loss: 0.0313, data_loss: 0.0313
step 522 , total_loss: 0.0246, data_loss: 0.0246
step 523 , total_loss: 0.0279, data_loss: 0.0279
step 524 , total_loss: 0.0325, data_loss: 0.0325
step 525 , total_loss: 0.0279, data_loss: 0.0279
step 526 , total_loss: 0.0314, data_loss: 0.0314
step 527 , total_loss: 0.0291, data_loss: 0.0291
step 528 , total_loss: 0.0272, data_loss: 0.0272
step 529 , total_los

step 679 , total_loss: 0.0135, data_loss: 0.0135
step 680 , total_loss: 0.0161, data_loss: 0.0161
step 681 , total_loss: 0.0182, data_loss: 0.0182
step 682 , total_loss: 0.0152, data_loss: 0.0152
step 683 , total_loss: 0.0174, data_loss: 0.0174
step 684 , total_loss: 0.0188, data_loss: 0.0188
step 685 , total_loss: 0.0131, data_loss: 0.0131
step 686 , total_loss: 0.0180, data_loss: 0.0180
step 687 , total_loss: 0.0158, data_loss: 0.0158
step 688 , total_loss: 0.0130, data_loss: 0.0130
step 689 , total_loss: 0.0149, data_loss: 0.0149
step 690 , total_loss: 0.0168, data_loss: 0.0168
step 691 , total_loss: 0.0131, data_loss: 0.0131
step 692 , total_loss: 0.0128, data_loss: 0.0128
step 693 , total_loss: 0.0177, data_loss: 0.0177
step 694 , total_loss: 0.0139, data_loss: 0.0139
step 695 , total_loss: 0.0134, data_loss: 0.0134
step 696 , total_loss: 0.0132, data_loss: 0.0132
step 697 , total_loss: 0.0121, data_loss: 0.0121
step 698 , total_loss: 0.0114, data_loss: 0.0114
step 699 , total_los

step 849 , total_loss: 0.0086, data_loss: 0.0086
step 850 , total_loss: 0.0076, data_loss: 0.0076
step 851 , total_loss: 0.0075, data_loss: 0.0075
step 852 , total_loss: 0.0082, data_loss: 0.0082
step 853 , total_loss: 0.0078, data_loss: 0.0078
step 854 , total_loss: 0.0089, data_loss: 0.0089
step 855 , total_loss: 0.0089, data_loss: 0.0089
step 856 , total_loss: 0.0079, data_loss: 0.0079
step 857 , total_loss: 0.0077, data_loss: 0.0077
step 858 , total_loss: 0.0075, data_loss: 0.0075
step 859 , total_loss: 0.0078, data_loss: 0.0078
step 860 , total_loss: 0.0096, data_loss: 0.0096
step 861 , total_loss: 0.0071, data_loss: 0.0071
step 862 , total_loss: 0.0088, data_loss: 0.0088
step 863 , total_loss: 0.0088, data_loss: 0.0088
step 864 , total_loss: 0.0072, data_loss: 0.0072
step 865 , total_loss: 0.0086, data_loss: 0.0086
step 866 , total_loss: 0.0082, data_loss: 0.0082
step 867 , total_loss: 0.0080, data_loss: 0.0080
step 868 , total_loss: 0.0076, data_loss: 0.0076
step 869 , total_los

step 1016 , total_loss: 0.0060, data_loss: 0.0060
step 1017 , total_loss: 0.0057, data_loss: 0.0057
step 1018 , total_loss: 0.0047, data_loss: 0.0047
step 1019 , total_loss: 0.0063, data_loss: 0.0063
step 1020 , total_loss: 0.0054, data_loss: 0.0054
step 1021 , total_loss: 0.0051, data_loss: 0.0051
step 1022 , total_loss: 0.0049, data_loss: 0.0049
step 1023 , total_loss: 0.0056, data_loss: 0.0056
step 1024 , total_loss: 0.0055, data_loss: 0.0055
step 1025 , total_loss: 0.0051, data_loss: 0.0051
step 1026 , total_loss: 0.0055, data_loss: 0.0055
step 1027 , total_loss: 0.0059, data_loss: 0.0059
step 1028 , total_loss: 0.0046, data_loss: 0.0046
step 1029 , total_loss: 0.0065, data_loss: 0.0065
step 1030 , total_loss: 0.0057, data_loss: 0.0057
step 1031 , total_loss: 0.0059, data_loss: 0.0059
step 1032 , total_loss: 0.0054, data_loss: 0.0054
step 1033 , total_loss: 0.0050, data_loss: 0.0050
step 1034 , total_loss: 0.0053, data_loss: 0.0053
step 1035 , total_loss: 0.0074, data_loss: 0.0074


step 1180 , total_loss: 0.0051, data_loss: 0.0051
step 1181 , total_loss: 0.0038, data_loss: 0.0038
step 1182 , total_loss: 0.0046, data_loss: 0.0046
step 1183 , total_loss: 0.0037, data_loss: 0.0037
step 1184 , total_loss: 0.0045, data_loss: 0.0045
step 1185 , total_loss: 0.0051, data_loss: 0.0051
step 1186 , total_loss: 0.0038, data_loss: 0.0038
step 1187 , total_loss: 0.0046, data_loss: 0.0046
step 1188 , total_loss: 0.0051, data_loss: 0.0051
step 1189 , total_loss: 0.0047, data_loss: 0.0047
step 1190 , total_loss: 0.0040, data_loss: 0.0040
step 1191 , total_loss: 0.0036, data_loss: 0.0036
step 1192 , total_loss: 0.0041, data_loss: 0.0041
step 1193 , total_loss: 0.0035, data_loss: 0.0035
step 1194 , total_loss: 0.0052, data_loss: 0.0052
step 1195 , total_loss: 0.0036, data_loss: 0.0036
step 1196 , total_loss: 0.0038, data_loss: 0.0038
step 1197 , total_loss: 0.0043, data_loss: 0.0043
step 1198 , total_loss: 0.0037, data_loss: 0.0037
step 1199 , total_loss: 0.0041, data_loss: 0.0041


KeyboardInterrupt: 

In [ ]:
res_syn = model.run_eval(test_file, num_ngs=test_num_ngs)
print(res_syn)

In [ ]:
model_best_trained = SeqModel(hparams, input_creator, seed=RANDOM_SEED)
path_best_trained = os.path.join(hparams.MODEL_DIR, "best_model")
print('loading saved model in {0}'.format(path_best_trained))
model_best_trained.load_model(path_best_trained)

In [ ]:
'''
dice + op
{'auc': 0.8221,
 'logloss': 1.0988,
 'mean_mrr': 0.6528,
 'ndcg@2': 0.5977,
 'ndcg@4': 0.6752,
 'ndcg@6': 0.708,
 'group_auc': 0.8229}
 
dice + op + disable bn
{'auc': 0.8192,
 'logloss': 1.1587,
 'mean_mrr': 0.6455,
 'ndcg@2': 0.5885,
 'ndcg@4': 0.6685,
 'ndcg@6': 0.7026, 
 'group_auc': 0.8199}

'''


model_best_trained.run_eval(test_file, num_ngs=test_num_ngs)